# Deep Water Bathymetric Particle Filter Experiment

This notebook runs the experiment testing the capabilities of the particle filter to conduct navigation using deep water bathymetry and for long duration.

## Data set preparation

First need to process the .m77t files in `source_data` into our database format.

In [1]:
from src.data_managment import m77t as tbx
from src.data_managment import dbmgr
import os
import pandas as pd
import numpy as np

In [2]:
data = tbx.read_m77t('./test/test_data.m77t')
data = tbx.m77t_to_df(data)

In [3]:
ins = tbx.create_trajectory(data)
dt = ins.index.to_series().diff().dt.total_seconds().fillna(0)
pds = tbx.find_periods((dt > 60).to_list())
trajs = tbx.split_dataset(ins, pds)

In [4]:
filepath:str = os.path.join('.db')
filename:str = 'sources.db'

In [5]:
db: dbmgr.DatabaseManager = dbmgr.DatabaseManager(source=os.path.join(filepath, filename))

In [6]:
db.get_trajectory(1)

,id,trajectory_id,timestamp,lat,lon,alt,roll,pitch,heading,speed,...,gyro_y,gyro_z,accel_x,accel_y,accel_z,depth,mag_tot,mag_res,gra_obs,freeair
0,1,1,1973-04-26 04:07:00,37.50166,-74.21333,0.0,0.0,0.0,71.529557,7.976108,...,-0.000056,-0.000045,0.077385,0.031718,-9.797804,1157.0,None,None,None,None
1,2,1,1973-04-26 04:08:00,37.50333,-74.20833,0.0,0.0,0.0,71.529557,7.975956,...,-0.000055,-0.002025,-0.000056,-0.000715,-9.798631,1148.0,None,None,None,None
2,3,1,1973-04-26 04:09:00,37.50500,-74.20333,0.0,0.0,0.0,63.502320,5.782016,...,-0.000056,0.000869,-0.071878,-0.044040,-9.798865,1144.0,None,None,None,None
3,4,1,1973-04-26 04:10:00,37.50666,-74.20000,0.0,0.0,0.0,71.529556,7.975654,...,-0.000052,-0.001720,0.074653,0.055329,-9.798882,1146.0,None,None,None,None
4,5,1,1973-04-26 04:11:00,37.50833,-74.19500,0.0,0.0,0.0,56.334659,9.597610,...,-0.000055,-0.000513,0.042095,-0.107682,-9.798489,970.0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,1189,1,1973-04-26 23:55:00,39.63000,-68.64333,0.0,0.0,0.0,71.632584,7.771435,...,-0.000054,-0.002576,0.001492,0.000178,-9.800258,2610.0,None,None,None,None
1189,1190,1,1973-04-26 23:56:00,39.63166,-68.63833,0.0,0.0,0.0,63.364493,5.671705,...,-0.000053,0.000495,-0.076904,-0.039700,-9.800893,NaN,None,None,None,None
1190,1191,1,1973-04-26 23:57:00,39.63333,-68.63500,0.0,0.0,0.0,71.529425,7.778474,...,-0.000053,0.000222,0.098817,0.032400,-9.800674,NaN,None,None,None,None
1191,1192,1,1973-04-26 23:58:00,39.63500,-68.63000,0.0,0.0,0.0,63.570684,5.673408,...,-0.000053,-0.001485,-0.082570,-0.041107,-9.800555,NaN,None,None,None,None


In [7]:
db.get_all_trajectories()

,id,source,start,stop,duration,distance,depth,mag_tot,mag_res,gra_obs,freeair,points
0,1,test_data,1973-04-26 04:07:00,1973-04-26 23:59:00,71520.0,546951.474873,True,False,False,False,False,1193
1,2,test_data,1973-04-27 00:02:00,1973-04-27 18:00:00,64680.0,305287.821952,True,False,False,False,False,1079
2,3,test_data,1973-04-27 22:14:00,1973-04-27 23:59:00,6300.0,28708.583342,True,False,False,False,False,106
3,4,test_data,1973-04-28 00:02:00,1973-04-28 01:00:00,3480.0,15966.057413,True,False,False,False,False,59
4,5,test_data,1973-04-28 05:21:00,1973-04-28 23:59:00,67080.0,386457.132840,True,False,False,False,False,1119
...,...,...,...,...,...,...,...,...,...,...,...,...
103,104,test_data,1973-07-04 15:46:00,1973-07-04 23:59:00,29580.0,151889.918467,True,True,True,False,False,494
104,105,test_data,1973-07-05 00:02:00,1973-07-05 23:59:00,86220.0,462348.415355,True,True,True,False,False,1438
105,106,test_data,1973-07-06 00:02:00,1973-07-06 21:16:00,76440.0,399897.222517,True,True,True,False,False,1275
106,107,test_data,1973-07-06 22:11:00,1973-07-06 23:59:00,6480.0,31866.874649,True,True,True,False,False,109


In [10]:
s = "/path/to/my/file.m77t"
s.split("/")[:-1]

['', 'path', 'to', 'my']

In [12]:
os.path.split(s)[1].split(".")[0]

'file'

### Preprocess tracklines into sql/df format

In [ ]:
# Check and see if the .db directory exists
if not os.path.exists(".db"):
    os.mkdir(".db")

# Check to see if the database exists
if not os.path.exists(".db/tracklines.db"):
    tables = []
else:
    tables = db.get_tables(".db/tracklines.db")

source_data_location = "./source_data/"
# walk through the .m77t files in the source_data directory
for root, dirs, files in os.walk(source_data_location):
    for file in files:
        if file.endswith(".m77t"):
            # check to see if the file has already been processed
            filename = os.path.splitext(file)[0]
            if filename not in tables:
                print("Processing file: " + file)
                data = pd.read_csv(os.path.join(root, file), delimiter="\t", header=0)
                data = tbx.m77t_to_df(data)
                # data.to_sql(
                #    filename, sqlite3.connect(".db/tracklines.db"), if_exists="replace"
                # )
                tbx.save_dataset(
                    [data],
                    [filename],
                    output_location=".db",
                    output_format="db",
                    dataset_name="tracklines",
                )
            else:
                print("Skipping file: " + file + " (already processed)")

### Parse the raw data into tracklines of continuous data collections

In [ ]:
max_time = 10  # minutes
max_delta_t = 2  # minutes between points
min_duration = 60  # minutes, minimum duration for a useful trackline

data, names = pdset.parse_tracklines_from_db(
    ".db/tracklines.db",
    # max_time,
    # max_delta_t,
    # min_duration,
    data_types=[
        "bathy",
        "mag",
        "grav",
        ["bathy", "mag"],
        ["bathy", "grav"],
        ["grav", "mag"],
        ["bathy", "grav", "mag"],
    ],
)
# Save the parsed data to the database
pdset.save_dataset(data, names, output_location=".db", output_format="db", dataset_name="parsed")
# summary = pdset.get_parsed_data_summary(data, names)

## Simulation parameters verification

First we need to tune the particle filter propagation noise to be similar to that of a marine-grade inertial navigation system. A low-end marine-grade INS should have a drift of 1 nm per 24 hours.

In [ ]:
from src.particle_filter import rmse, propagate
import numpy as np

time = 24 * 60  # minutes
noise = np.array([0, 2.6, 0])
bound = 1852  # meters

errors = []
for v in range(1, 26):
    P = np.asarray([[0, 0, 0, 0, v, 0]])
    T = P.copy()
    t = 0
    for i in range(50000):
        # Eastward
        u = [0, v, 0]
        while t < time:
            P = propagate(P, u, noise=np.diag(noise), noise_calibration_mode=True)
            T = propagate(T, u, noise=np.diag([0, 0, 0]), noise_calibration_mode=False)
            t += 1
        errors.append(rmse(P, T[0, :2]))
        # Northward
        u = [v, 0, 0]
        while t < time:
            P = propagate(P, u, noise=np.diag(noise), noise_calibration_mode=True)
            T = propagate(T, u, noise=np.diag([0, 0, 0]), noise_calibration_mode=False)
            t += 1
        errors.append(rmse(P, T[0, :2]))
        # Northeastward
        u = np.array([1, 1, 0]) / np.linalg.norm([1, 1, 0])
        u *= v
        while t < time:
            P = propagate(P, u, noise=np.diag(noise), noise_calibration_mode=True)
            T = propagate(T, u, noise=np.diag([0, 0, 0]), noise_calibration_mode=False)
            t += 1
        errors.append(rmse(P, T[0, :2]))

print(f"RMSE: {np.mean(errors)}")

In [ ]:
from matplotlib import pyplot as plt

if not os.path.exists(".db/plots"):
    os.makedirs(".db/plots")

plt.hist(errors, bins=15, density=True)
plt.xlabel("RMSE (m)")
plt.ylabel("Frequency")
plt.title("RMSE of Particle Filter")
plt.savefig(".db/plots/propagation_tuning.png")
plt.show()

In [ ]:
config = {"velocity_noise": [noise[1], noise[1], 0]}

## Develop measurement model

Next we need to develop the measurement value standard deviation. We'll first do some general examination of the data. Namely, investigating the sensor measurements to see if we can build a reasonable sensor model.

In [ ]:
from src.gmt_tool import inflate_bounds, get_map_section, get_map_point
import numpy as np
import json

if os.path.exists("config.json"):
    config = json.load(open("config.json", "r"))

tables = pdset.get_tables(".db/parsed.db")
bathy_tables = [table for table in tables if "_D_" in table]

d_bathy = np.array([])

for table in bathy_tables:
    data = pdset.table_to_df(".db/parsed.db", table)
    min_lon = data.LON.min()
    max_lon = data.LON.max()
    min_lat = data.LAT.min()
    max_lat = data.LAT.max()
    min_lon, min_lat, max_lon, max_lat = inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    bathy_map = get_map_section(min_lon, max_lon, min_lat, max_lat, "relief", "15s", "temp")
    d_bathy = np.hstack([d_bathy, data["DEPTH"] - (-get_map_point(bathy_map, data.LON, data.LAT))])

config["bathy_mean_d"] = np.mean(d_bathy, where=~np.isnan(d_bathy))
config["bathy_std"] = np.std(d_bathy, where=~np.isnan(d_bathy))

if os.path.exists("config.json"):
    # delete the file
    os.remove("config.json")

with open("config.json", "w") as f:
    json.dump(config, f)

In [ ]:
plt.hist(d_bathy, bins=200, density=True)
plt.xlim([-300, 300])
plt.xlabel("Depth Difference (m)")
plt.ylabel("Frequency")
plt.title("Bathymetry Difference")
plt.savefig(".db/plots/bathy_diff.png")
plt.show()

In [ ]:
config

In [ ]:
import json

with open("config.json", "w") as f:
    json.dump(config, f)

### Populate with velocities

In [ ]:
from src.particle_filter import (
    process_particle_filter,
    populate_velocities,
    plot_error,
    plot_estimate,
    summarize_results,
)
import json
import src.process_dataset as pdset
from tqdm import tqdm
import pandas as pd
import os
import matplotlib.pyplot as plt

In [ ]:
tables = pdset.get_tables(".db/parsed.db")
bathy_tables = [table for table in tables if "_D_" in table]
config = json.load(open("config.json", "r"))
config["n"] = 1000
config["cov"] = [
    1 / 60,
    1 / 60,
    0,
    config["velocity_noise"][0],
    config["velocity_noise"][1],
    0,
]

with open("config.json", "w") as f:
    json.dump(config, f)

In [ ]:
config

In [ ]:
df = pdset.table_to_df(".db/parsed.db", bathy_tables[3])

In [ ]:
len(df)

In [ ]:
if not os.path.exists(".db/plots2/estimate/"):
    os.makedirs(".db/plots2/estimate/")

if not os.path.exists(".db/plots2/errors/"):
    os.makedirs(".db/plots2/errors/")

summary = None
annotations = {"recovery": 1852, "res": 1852 / 4}
# for table in tqdm(bathy_tables):
#     print(f"Running {table}")
#     df = pdset.table_to_df(".db/parsed.db", table)

df = populate_velocities(df)
results, geo_map = process_particle_filter(df, config)
print("Run complete! Saving results...")
pdset.save_dataset(
    [results],
    ["test"],
    output_location=".db",
    output_format="db",
    dataset_name="results",
)

In [ ]:
print("Results saved! Plotting...")
fig, ax = plot_estimate(geo_map, results)
fig.savefig(f".db/plots2/estimate/test_estimate.png")
plt.close(fig)
fig, ax = plot_error(results, annotations=annotations)
fig.savefig(f".db/plots2/errors/test_error.png")
plt.close(fig)

In [ ]:
summary = None
results_tables = pdset.get_tables(".db/results.db")
for table in results_tables:
    df = pdset.table_to_df(".db/results.db", table)
    run = summarize_results(df, 1852)
    run["Name"] = table
    if summary is None:
        summary = run  # .copy()
    else:
        summary = pd.concat([summary, run], ignore_index=True)
    summary.to_csv(".db/summary_recovery.csv")

In [ ]:
summary = None
results_tables = pdset.get_tables(".db/results.db")
for table in results_tables:
    df = pdset.table_to_df(".db/results.db", table)
    run = summarize_results(df, 452)
    run["Name"] = table
    if summary is None:
        summary = run  # .copy()
    else:
        summary = pd.concat([summary, run], ignore_index=True)
    summary.to_csv(".db/summary_resolution.csv")

# Data Post Processing

Use this section to load and post process the results data set

In [ ]:
import src.process_dataset as pdset
import pandas as pd
import matplotlib.pyplot as plt
from src.particle_filter import summarize_results
import os
from datetime import timedelta

In [ ]:
results_tables = pdset.get_tables(".db/results.db")
len(results_tables)

In [ ]:
for table in results_tables:
    table_df = pdset.table_to_df(".db/results.db", table)
    summary = summarize_results(table, table_df, 1852)
    summary.to_csv(
        ".db/plots/summary.csv",
        mode="a",
        header=(not os.path.exists(".db/plots/summary.csv")),
    )

In [ ]:
summary = pd.read_csv(
    ".db/plots/summary.csv",
    header=0,
    dtype={
        "": int,
        "name": str,
        "start": str,
        "stop": str,
        "duration": str,
        "average_error": float,
        "max_error": float,
        "min_error": float,
    },
)
summary["num"] = summary["Unnamed: 0"]
summary = summary.drop(columns=["Unnamed: 0"])
# summary['start'] = pd.to_datetime(summary['start'], format="%Y-%m-%d %H:%M:%S%z")
# summary['end'] = pd.to_datetime(summary['end'], format="%Y-%m-%d %H:%M:%S%z")
summary["start"] = pd.to_timedelta(summary["start"])
summary["end"] = pd.to_timedelta(summary["end"])
summary["duration"] = pd.to_timedelta(summary["duration"])
summary.head()

len(summary)

In [ ]:
summary.head()

In [ ]:
recovery = summary.loc[summary["min error"] > 452]
len(recovery)

In [ ]:
# check to see if all the tables in results_tables are present in summary["name"] and if not capture the missing tables
missing = []
for table in results_tables:
    if table not in summary["name"].values:
        missing.append(table)

total = len(results_tables)
num_recoveries = total - len(missing)
print(
    f"There are {total} total trajectories. We were able to recover at least one position fix below drift error in {num_recoveries} ({num_recoveries / total :0.4f}) trajectories."
)

In [ ]:
pixel = summary.loc[summary["min error"] <= 452]

# check to see if the tables in pixel are present in summary["name"] and if not capture the missing tables
missing = []
for table in results_tables:
    if table not in pixel["name"].values:
        missing.append(table)
below_pixel_fixes = total - len(missing)

print(
    f"There are {len(pixel)} total below pixel resolution fixes. We were able to achieve at least one position estimate below drift error in {below_pixel_fixes} ({below_pixel_fixes/total :0.4f}) trajectories."
)
print(f"mean duration: {pixel['duration'].mean()} and median duration: {pixel['duration'].median()}")
print(f"mean error: {pixel['min error'].mean()} and median error: {pixel['min error'].median()}")
print(f"minium duration: {pixel['duration'].min()} and maximum duration: {pixel['duration'].max()}")
print(f"minimum error: {pixel['min error'].min()} and maximum error: {pixel['min error'].max()}")
print(f"mean start: {pixel['start'].mean()} and median start: {pixel['start'].median()}")

In [ ]:
summary.sort_values(by="min error").head(3)

In [ ]:
summary.sort_values(by="start").head(3)

In [ ]:
summary.sort_values(by="duration").tail(3)

In [ ]:
summary.sort_values(by="average_error").head(3)

In [ ]:
# Find the line in summary that has the closest to the mean duration
summary.loc[abs(summary["duration"] - summary["duration"].median()) <= timedelta(minutes=1)]

In [ ]:
summary["duration"].median()

In [ ]:
print(
    f"On average we were able to recover a position fix with an mean duration of {summary['duration'].mean()}, median duration of {summary['duration'].median()} and a mean error of {summary['average_error'].mean()} and median error {summary['average_error'].median()}."
)

print(f"Minimum duration {summary['duration'].min()} and maximum duration {summary['duration'].max()}.")
print(f"Minimum error {summary['average_error'].min()} and maximum error {summary['average_error'].max()}.")

In [ ]:
first = summary.loc[summary["num"] == 0]
# first.head()

print(
    f"The first position recover occurs with a mean of {first['start'].mean()} and median {first['start'].median()} after the start of the trajectory."
)
print(
    f"with an mean duration of {first['duration'].mean()}, median duration of {first['duration'].median()} and a mean error of {first['average_error'].mean()} and median error {first['average_error'].median()}."
)

print(f"Minimum duration {first['duration'].min()} and maximum duration {first['duration'].max()}.")
print(f"Minimum error {first['average_error'].min()} and maximum error {first['average_error'].max()}.")

# Gravity

Recreate the above simulation and measurment model development this time with gravity

In [ ]:
from src.gmt_tool import inflate_bounds, get_map_section, get_map_point
import src.process_dataset as pdset
import numpy as np
import json
from matplotlib import pyplot as plt
import os

config = json.load(open("config.json", "r"))
tables = pdset.get_tables(".db/parsed.db")
gravity_tables = [table for table in tables if "_G_" in table]

In [ ]:
d_gravity = np.array([])

for table in gravity_tables:
    data = pdset.table_to_df(".db/parsed.db", table)
    min_lon = data.LON.min()
    max_lon = data.LON.max()
    min_lat = data.LAT.min()
    max_lat = data.LAT.max()
    min_lon, min_lat, max_lon, max_lat = inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    gravity_map = get_map_section(min_lon, max_lon, min_lat, max_lat, "gravity", "01m", "temp")
    d_gravity = np.hstack([d_gravity, data["GRAV_ANOM"] - get_map_point(gravity_map, data.LON, data.LAT)])

config["gravity_mean_d"] = np.mean(d_gravity, where=~np.isnan(d_gravity))
config["gravity_std"] = np.std(d_gravity, where=~np.isnan(d_gravity))

if os.path.exists("config.json"):
    # delete the file
    os.remove("config.json")

with open("config.json", "w") as f:
    json.dump(config, f)

plt.hist(d_gravity, bins=100, density=True)
plt.xlim([-50, 75])
plt.xlabel("Gravity Difference (mGal)")
plt.ylabel("Frequency")
plt.title("Gravity Anomaly Difference")
plt.savefig(".db/plots/gravity_diff.png")
plt.show()

# Magnetics

Recreation with magnetics

In [ ]:
from src.gmt_tool import inflate_bounds, get_map_section, get_map_point
import numpy as np
import json

config = json.load(open("config.json", "r"))
tables = pdset.get_tables(".db/parsed.db")
mag_tables = [table for table in tables if "_M_" in table]

df = pdset.table_to_df(".db/parsed.db", mag_tables[0])
df.head()

In [ ]:
d_magnetics = np.array([])

for table in mag_tables:
    data = pdset.table_to_df(".db/parsed.db", table)
    min_lon = data.LON.min()
    max_lon = data.LON.max()
    min_lat = data.LAT.min()
    max_lat = data.LAT.max()
    min_lon, min_lat, max_lon, max_lat = inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    mag_map = get_map_section(min_lon, max_lon, min_lat, max_lat, "magnetic", "02m", "temp")
    d_magnetics = np.hstack([d_magnetics, data["MAG_RES"] - get_map_point(mag_map, data.LON, data.LAT)])

config["magnetic_mean_d"] = np.mean(d_magnetics, where=~np.isnan(d_magnetics))
config["magnetic_std"] = np.std(d_magnetics, where=~np.isnan(d_magnetics))

if os.path.exists("config.json"):
    # delete the file
    os.remove("config.json")

with open("config.json", "w") as f:
    json.dump(config, f)

In [ ]:
plt.hist(d_magnetics, bins=100, density=True)
plt.xlim([-500, 500])
plt.xlabel("Magnetic Difference (nT)")
plt.ylabel("Frequency")
plt.title("Magnetic Residual Difference")
plt.savefig(".db/plots/mag_diff.png")
plt.show()

In [ ]:
if not "":
    print("EMPTY")

In [ ]:
st = [str(i) for i in range(10)]


In [ ]:
ST = "".join(st)

In [ ]:
ST

In [ ]:
"A" in "AaBbCc"

In [ ]:
trx, names = tbx.parse_trackline_from_file("./test/test_data.csv")

In [ ]:
names

In [ ]:
if not []:
    print("EMPTY")

In [ ]:
"G" in "DMG"

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv("78123006.csv")

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(
        "./test/test_data.m77t",
        header=0,
        index_col=0,
        parse_dates=True,
        dtype={
            "LAT": float,
            "LON": float,
            "CORR_DEPTH": float,
            "MAG_TOT": float,
            "MAG_RES": float,
            "GRA_OBS": float,
            "FREEAIR": float,
        },
    )

In [ ]:
df.head()

In [ ]:
df.drop("AB")

In [ ]:
from src.data_manager import m77t as tbx
import pandas as pd

In [ ]:
data = tbx.read_m77t("./source_data/78123006.m77t")
data = tbx.m77t_to_df(data)

In [ ]:
schema = pd.io.json.build_table_schema(data)
schema

In [ ]:
from sqlalchemy import create_engine

def get_engine(server, database, username, password):
    """Creates and returns an SQLAlchemy engine for SQL Server."""
    connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
    engine = create_engine(connection_string)
    return engine


In [ ]:
from sqlalchemy import Table, Column, Integer, String, MetaData, select
from sqlalchemy.dialects.mssql import NVARCHAR, FLOAT

metadata = MetaData()

# Define your table schema as reflected from the database or defined by your application
collections = Table('Collections', metadata,
                    Column('CollectionID', Integer, primary_key=True),
                    Column('CollectionName', NVARCHAR(255), nullable=False),
                    Column('Description', NVARCHAR('max'))
                    )

def insert_collection(engine, collection_name, description=""):
    """Insert a new collection using SQLAlchemy and return its CollectionID."""
    insert_stmt = collections.insert().values(CollectionName=collection_name, Description=description)
    with engine.connect() as conn:
        result = conn.execute(insert_stmt)
        conn.commit()
        collection_id = result.inserted_primary_key[0]  # Get the primary key of the inserted record
    return collection_id

def upload_data(df, collection_id, engine):
    """Uploads data from a DataFrame to the RawData table, linking it to the specified CollectionID."""
    df['CollectionID'] = collection_id  # Add CollectionID to the DataFrame
    df.to_sql(name='RawData', con=engine, if_exists='append', index=False)

def query_data_by_collection(collection_id, engine):
    """Queries data for a specific collection and returns it as a DataFrame."""
    query = f"SELECT * FROM RawData WHERE CollectionID = {collection_id}"
    return pd.read_sql_query(query, con=engine)



In [ ]:
data.head()

In [ ]:
engine = get_engine("localhost", "geonavdb", "sa", "2016SpringGardenStreet!")

In [ ]:
col_id = insert_collection(engine, "test_collection", "This is a test collection")